### Machine Learning models. 

In this process we will use some machine learning models in order to see how is the performance on each one. 

In [1]:
import gradio

ModuleNotFoundError: No module named 'dash'

In [7]:
import pandas as pd 
import numpy as np
import seaborn as sns

from ml_models import ml_models


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline

#### Data ready to go.

In [8]:
# Open the HDF5 file
with pd.HDFStore('data_etl.h5', mode='r') as store:
    # Get the keys (structure) within the HDF5 file
    keys = store.keys()
# Display the keys
print("Keys in the HDF5 file:")
for key in keys:
    print(key)

Keys in the HDF5 file:
/X_test
/X_test_pca
/X_train
/X_train_pca
/y_test
/y_train


In [9]:
with pd.HDFStore('data_etl.h5', mode='r') as store:
    # Get the keys (structure) within the HDF5 file
    keys = store.keys()
    # Access and retrieve specific DataFrames
    X_train = store.get('/X_train')
    X_test = store.get('/X_test')
    X_train_pca = store.get('/X_train_pca')
    X_test_pca = store.get('/X_test_pca')
    y_train = store.get('/y_train')
    y_test = store.get('/y_test')

#### Creation of a Pipeline. 

In [14]:
# Assuming X_train and y_train are your training data

# Extract numeric and non-numeric column names
numeric_columns = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
non_numeric_columns = X_train.select_dtypes(exclude=['float64', 'int64']).columns.tolist()

# Define the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numeric_columns),
        # Add other transformations as needed (e.g., encoding categorical variables)
        # ('imputer', SimpleImputer(strategy='mean'), numeric_columns),  # If there are missing values
    ],
    remainder='passthrough'  # Pass through columns not listed for transformation
)

# Define classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Neural Network': MLPClassifier(max_iter=1000)
}

# Create a pipeline for each classifier
pipelines = {}
for clf_name, clf in classifiers.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', clf)
    ])
    pipelines[clf_name] = pipeline

# Set hyperparameters grid for GridSearchCV
param_grids = {
    'Logistic Regression': {'classifier__C': [0.1, 1.0, 10.0]},
    'Random Forest': {'classifier__n_estimators': [100, 200, 300]},
    'Neural Network': {'classifier__hidden_layer_sizes': [(64,), (128,), (64, 32)]}
}

# Perform GridSearchCV to find the best model
best_models = {}
for clf_name, pipeline in pipelines.items():
    grid_search = GridSearchCV(pipeline, param_grid=param_grids[clf_name], cv=5)
    grid_search.fit(X_train, y_train)
    best_models[clf_name] = grid_search.best_estimator_

# Evaluate and choose the best model
for clf_name, best_model in best_models.items():
    print(f"Best {clf_name} model: {best_model.score(X_train, y_train)}")

# Select the best model based on validation performance or other criteria
best_model = best_models['Logistic Regression']  # Change to the best-performing model

# Use best_model for predictions or further analysis

c:\Users\luisa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\luisa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\luisa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\luisa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1183: DataConve